In [23]:
import pandas as pd
import os

In [ ]:
df_bio = pd.read_pickle('../data_raw/df_bio.pkl')
df_condition = pd.read_pickle('../data_raw/df_condition.pkl')
df_dedup_deterministic = pd.read_pickle('../data_raw/df_dedup_deterministic.pkl')
df_dedup_proba = pd.read_pickle('../data_raw/df_dedup_proba.pkl')
df_note = pd.read_pickle('../data_raw/df_note.pkl')
df_person = pd.read_pickle('../data_raw/df_person.pkl')
df_visit = pd.read_pickle('../data_raw/df_visit.pkl')

**df_person**

In [57]:
# Le CDM est identique pour tous les patients, on le supprime
assert(len(df_person['cdm_source'].unique()) == 1)
df_person_clean = df_person.drop(columns=['cdm_source'])

# On supprime les patients sans date de naissance, ou avec une date de naissance trop ancienne (incoherent)
df_person_clean = df_person_clean[df_person_clean['birth_datetime'] > '1920-01-01']

# Transform the gender source value that are equal to f to female and m to male
df_person_clean['gender_source_value'] = df_person_clean['gender_source_value'].replace({'f': 'female', 'm': 'male'})
assert(len(df_person_clean.gender_source_value.unique()) == 2)

# Remove male patients
df_person_clean = df_person_clean[df_person_clean['gender_source_value'] == 'female']
assert(len(df_person_clean.gender_source_value.unique()) == 1)

# Save it as a pickle to dara_clean
output_dir = '../data_clean'
os.makedirs(output_dir, exist_ok=True)

# Sauvegarder le DataFrame nettoyé sous forme de pickle dans data_clean
output_path = os.path.join(output_dir, 'df_person.pkl')
df_person_clean.to_pickle(output_path)

**df_bio**

In [56]:
df_bio_clean = df_bio

output_path = os.path.join(output_dir, 'df_bio.pkl')
df_bio_clean.to_pickle(output_path)

In [55]:
df_visit_clean = df_visit[df_visit['person_id'].isin(df_person_clean['person_id'])]

output_path = os.path.join(output_dir, 'df_visit.pkl')
df_visit_clean.to_pickle(output_path)

In [ ]:
df_note_clean = df_note.drop(columns=['cdm_source'])
df_note_clean = df_note_clean[df_note_clean['visit_occurrence_id'].isin(df_visit_clean['visit_occurrence_id'])]

df_note_clean.info()

output_path = os.path.join(output_dir, 'df_note.pkl')
df_note_clean.to_pickle(output_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   visit_occurrence_id  1547 non-null   float64       
 1   note_datetime        1547 non-null   datetime64[ns]
 2   note_id              1547 non-null   float64       
 3   cdm_source           1547 non-null   object        
 4   note_text            1547 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 60.6+ KB
